<a href="https://colab.research.google.com/github/LamdaMC/Mandar_correos_automaticamente_con_excel/blob/main/Mandar_correo_automaticamente.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Este programa inicialmente se hizo en visualstudiocode por eso algunas cosas no funcionan bien estoy solcuonando es

#Correo

Para que el correo funcione primero se tiene que activar la verificacion en dos pasos
aqui esta el link
https://myaccount.google.com/security

despues hecho eso se tiene que obtener la clave en la contraseña de aplicaciones
https://myaccount.google.com/u/4/apppasswords
pones la clave en la parte que dice password y listo :D

#Excel
El archivo de Excel tiene que tener una terminacion xlsx ya que aun no configuro para mas entonrnos, ademas este mismo tiene que tener al menos dos columnas la de Nombre y Correo que es donde se buscara y obtendra la informacion

In [4]:
"""
__autor__ = Diego Omar Matias Cruz
__github__ = https://github.com/LamdaMC/Mandar_correos_automaticamente_con_excel

VERSION = 1.1
"""

import os
import subprocess
import mimetypes
import glob
import ssl
import smtplib
import pandas as pd
#from dotenv import load_dotenv #se descomente si estas en visualstudio
from email.message import EmailMessage

load_dotenv(".env")
"""
Se ocupa esta linea para abrir el archivo .env que es donde se guardaran el remitente, la contrase;a, el asunto y el cuerpo
"""

remitente = "experimentosdomc@gmail.com"
password = "juwuxuuwojrrxeab"
asunto = "Hola mundo"
cuerpo = "Arriba el el boing de mango"

class Buscarcorreos:
    """

    Buscarcorreos es una clase la cual busca los correos de una persona en concreto y se mandan los siguientes parametros

    :nombre: este es el nombre que se desea buscar de preferencia se usan cadenas de caracteres
    :ruta: este es donde guardamos nuestro excel donde guardamos los datos de las personas a las que se les envian el correo

    """

    def __init__(self,nombre,ruta):

       self.buscar_nombre = nombre
       self.ruta = ruta



    def buscar_encotrar(self):

        """
        Con la funcion buscar_encotrar lo que hace es recibir el parametro de la ruta del archivo excel
        donde esta guardado el nombre de las personas a las que les quieres mandar el correo y para usarla es asi

        Buscarcorreos("Nombre_a_buscar, Ruta_excel").buscar_encotrar()

        """


        df = pd.read_excel(self.ruta)
        correos = []

        for i in range(len(df["Nombre"])):
            if (pd.notna(df["Nombre"]).iloc[i] and pd.notna(df["Correo"]).iloc[i] and self.buscar_nombre in df["Nombre"].iloc[i]):
                nuevo_correo = df["Correo"].iloc[i]
                correos.append(nuevo_correo)
                print("Correo encontrado")

            if pd.notna(df["Correo"]).iloc[i]:
                print("No hay correo registrado a ese nombre")

        return correos

class Correo:
    """
    Esta clase lo que hace es que con los correos que se le manden la clase Buscarcorreos es enviar el archivo junto al correo ya obtenido

    :asunto: Este dato lo obtenemos del archivo .env que guardamos en la misma carpeta que este programa
    :cuerpo: Este dato lo obtenemos del archivo .env que guardamos en la misma carpeta que este programa
    :destinatario: Este archivo lo obtiene el programa de la clase BuscarCorreo
    :archivo: Este es el archivo que adjuntaremos para enviar
    :remitente: Este dato lo obtenemos del archivo .env que guardamos en la misma carpeta que este programa
    :password: Este dato lo obtenemos del archivo .env que guardamos en la misma carpeta que este programa

    """

    def __init__(self,asunto, cuerpo,destinatario, archivo,remitente,password):

        self.contrasena = password
        self.correo = remitente
        self.asunto = asunto
        self.cuerpo = cuerpo
        self.destinatario = destinatario
        self.archivo = archivo

    def obtener_tipo_archivo(self):

        tipo_archivo, encoding = mimetypes.guess_type(self.archivo)
        return tipo_archivo

    def enviarcorreo(self):
        em = EmailMessage()
        em["From"] = self.correo
        em["To"] = self.destinatario
        em["Subject"] = self.asunto
        em.set_content(self.cuerpo)
        tipo_ar = self.obtener_tipo_archivo()

        maintype = None
        subtype = None

        if tipo_ar == 'image/png':
            maintype = 'image'
            subtype = 'png'
            print("Es una imagen PNG.")

        elif tipo_ar == 'image/jpeg':
            maintype = 'image'
            subtype = 'jpeg'
            print("Es una imagen JPEG.")

        elif tipo_ar == 'application/pdf':
            maintype = 'application'
            subtype = 'pdf'
            print("Es un archivo PDF.")

        elif tipo_ar == 'text/plain':
            maintype = 'text'
            subtype = 'plain'
            print("Es un archivo de texto sin formato.")

        elif tipo_ar == 'audio/mpeg':
            maintype = 'audio'
            subtype = 'mpeg'
            print("Es un archivo de audio MP3.")

        elif tipo_ar == 'video/mp4':
            maintype = 'video'
            subtype = 'mp4'
            print("Es un archivo de video MP4.")

        elif tipo_ar == 'application/msword':
            maintype = 'application'
            subtype = 'msword'
            print("Es un archivo de Word (formato .doc).")

        elif tipo_ar == 'application/vnd.openxmlformats-officedocument.wordprocessingml.document':
            maintype = 'application'
            subtype = 'vnd.openxmlformats-officedocument.wordprocessingml.document'
            print("Es un archivo de Word (formato .docx).")

        elif tipo_ar == 'application/zip':
            maintype = 'application'
            subtype = 'zip'
            print("Es un archivo ZIP.")

        else:
            print("Tipo de archivo desconocido.")

        if maintype and subtype:
            with open(self.archivo, 'rb') as archivo_imagen:
                em.add_attachment(archivo_imagen.read(), maintype=maintype, subtype=subtype, filename=os.path.basename(self.archivo))
                print("Archivo adjunto agregado correctamente.")

            context = ssl.create_default_context()

            with smtplib.SMTP_SSL("smtp.gmail.com",465, context = context) as smtp:
                smtp.login(self.correo, self.contrasena)
                smtp.sendmail(self.correo, self.destinatario, em.as_string())
                print("Correo enviado con éxito.")


def install_dependencies():
    try:
        import pandas
        import dotenv
        import openpyxl
        import requests

        print("Las dependencias ya están instaladas.")

    except ImportError:
        try:
            subprocess.check_call(["pip", "install", "pandas", "python-dotenv", "openpyxl", "requests"])
            print("Dependencias instaladas correctamente.")
        except subprocess.CalledProcessError:
            print("Error al instalar dependencias.")

def main():
    nombremain = input("Escriba el nombre a buscar: ")
    archivomain = input("¿Como se llama el archivo que desea enviar?: ")
    excel_main = input("¿Como se llama el archivo donde tiene los correos?: ")

    correosmain  = Buscarcorreos(nombremain,excel_main).buscar_encotrar()

    for correo in correosmain:
        Correo(asunto,cuerpo,correo,archivomain,remitente,password).enviarcorreo()
        print("se enviaron los correos con exito")


if __name__ == "__main__":
    #install_dependencies() # Se quita el comentario si estas en vs
    main()

NameError: ignored